# Задание №1 (easy)

In [1]:
import pandas
import sklearn.model_selection
import sklearn.linear_model
import sklearn.metrics
import sklearn.neighbors
import sklearn.naive_bayes
import sklearn.svm

C:\Users\Artem\AppData\Local\Temp\ipykernel_12308\272673677.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [2]:
%%time

data = pandas.read_pickle('../data/train.pickle')
x, y = data['features'], data['labels']
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, train_size=0.8, random_state=10 ** 9 + 7)
model = sklearn.linear_model.LogisticRegression(max_iter=128)
model.fit(x_train.reshape(x_train.shape[0], -1), y_train)
y_pred = model.predict(x_test.reshape(x_test.shape[0], -1))
sklearn.metrics.precision_score(y_test, y_pred, average='weighted')

CPU times: total: 2min 21s
Wall time: 18.7 s


C:\Programs\Miniconda\envs\env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9347966439161737

Была выбрана такая метрика, так как она учитывает не общее количество неудач модели, а высчитывает средневзвешенное между точностей для каждой категории, что важно, ведь подобная модель должна с хорошей точностью опеределять все типы знаков, а не в общей сложности, так как если какие-то знаки встречаются реже, и модель плохо их распознаёт, то это критически недопустимая ситуация для, например, беспилотных автомоблей.

In [3]:
labels = pandas.read_pickle('../data/labels.pickle')

def show_worst_items():
    global labels, y_pred, y_test
    mistakes = {}
    for i in range(len(y_test)):
        if y_pred[i] != y_test[i]:
            if not (labels[y_test[i]] in mistakes):
                mistakes[labels[y_test[i]]] = 0
            mistakes[labels[y_test[i]]] += 1
    
    print(*sorted(list(mistakes.items()), key=lambda x : x[1], reverse=True)[:5], sep='\n')

show_worst_items()

('Speed limit (60km/h)', 40)
('Speed limit (50km/h)', 37)
('Speed limit (80km/h)', 36)
('Speed limit (30km/h)', 35)
('Speed limit (120km/h)', 34)


Можно заметить, что чаще всего модель ошибается в определении чисел, что неудивительно из-за качества изображений.

# Задание №2 (medium)

In [4]:
%%time

model = sklearn.neighbors.KNeighborsClassifier(n_neighbors=16)
model.fit(x_train.reshape(x_train.shape[0], -1), y_train)
print(sklearn.metrics.precision_score(y_test, model.predict(x_test.reshape(x_test.shape[0], -1)), average='weighted'))
show_worst_items()

0.7990170169629001
('Speed limit (60km/h)', 40)
('Speed limit (50km/h)', 37)
('Speed limit (80km/h)', 36)
('Speed limit (30km/h)', 35)
('Speed limit (120km/h)', 34)
CPU times: total: 43.9 s
Wall time: 10.1 s


In [5]:
%%time

model = sklearn.naive_bayes.MultinomialNB()
model.fit(x_train.reshape(x_train.shape[0], -1), y_train)
print(sklearn.metrics.precision_score(y_test, model.predict(x_test.reshape(x_test.shape[0], -1)), average='weighted'))
show_worst_items()

0.47788173327432193
('Speed limit (60km/h)', 40)
('Speed limit (50km/h)', 37)
('Speed limit (80km/h)', 36)
('Speed limit (30km/h)', 35)
('Speed limit (120km/h)', 34)
CPU times: total: 1min 4s
Wall time: 1min 4s


In [6]:
%%time

model = sklearn.svm.SVC(kernel='linear', max_iter=8)
model.fit(x_train.reshape(x_train.shape[0], -1), y_train)
print(sklearn.metrics.precision_score(y_test, model.predict(x_test.reshape(x_test.shape[0], -1)), average='weighted'))
show_worst_items()

C:\Programs\Miniconda\envs\env\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


0.5367096180069916
('Speed limit (60km/h)', 40)
('Speed limit (50km/h)', 37)
('Speed limit (80km/h)', 36)
('Speed limit (30km/h)', 35)
('Speed limit (120km/h)', 34)
CPU times: total: 1min 52s
Wall time: 1min 51s


In [7]:
model = sklearn.svm.SVC(kernel='poly', max_iter=8)
model.fit(x_train.reshape(x_train.shape[0], -1), y_train)
print(sklearn.metrics.precision_score(y_test, model.predict(x_test.reshape(x_test.shape[0], -1)), average='weighted'))
show_worst_items()

C:\Programs\Miniconda\envs\env\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


0.1625947614805924
('Speed limit (60km/h)', 40)
('Speed limit (50km/h)', 37)
('Speed limit (80km/h)', 36)
('Speed limit (30km/h)', 35)
('Speed limit (120km/h)', 34)


C:\Programs\Miniconda\envs\env\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
model = sklearn.svm.SVC(kernel='rbf', max_iter=8)
model.fit(x_train.reshape(x_train.shape[0], -1), y_train)
print(sklearn.metrics.precision_score(y_test, model.predict(x_test.reshape(x_test.shape[0], -1)), average='weighted'))
show_worst_items()

C:\Programs\Miniconda\envs\env\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


0.31580700396205613
('Speed limit (60km/h)', 40)
('Speed limit (50km/h)', 37)
('Speed limit (80km/h)', 36)
('Speed limit (30km/h)', 35)
('Speed limit (120km/h)', 34)


C:\Programs\Miniconda\envs\env\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Слабым местом KNN скорее всего стала крайная близость знаков ограничения скорости, что усугубляетя тем, что они являются слабым местом для всех моделей. НБК предполагает, что признаки являются независимыми, но многие из них связаны напрямую, например, все запрещающие знаки имеют круглую форму и красную обводку. Вероятнон чтоо, данные не могут быть разделенполиномиально (в том числе линейно)но, поэтому применениSVN даёт не лучший результат.шие.